In [1]:
import numpy as np
import time

In [2]:
import sys
import SimpleGraph as sg

## Test algorithm for finding strongly connected components

In [7]:
graph = sg.random_simple_graph(5, directed=True, prob=.3)
print('The graph:\n\t{}'.format(graph))

r_graph = sg.reverse(graph)
print('\nThe reversed graph:\n\t{}\n'.format(r_graph))

SCC = sg.stronglyConnectedComp(graph)
for i, scc in enumerate(SCC):
    print('SCC {}: {}'.format(i, scc))

The graph:
	{0: {}, 1: {3: 1.0, 4: 1.0}, 2: {4: 1.0}, 3: {0: 1.0}, 4: {2: 1.0}}

The reversed graph:
	{0: {3: 1.0}, 1: {}, 2: {4: 1.0}, 3: {1: 1.0}, 4: {1: 1.0, 2: 1.0}}

SCC 0: [1]
SCC 1: [2, 4]
SCC 2: [3]
SCC 3: [0]


## Tests of shortest path distance algorithm

### Test 1: small graph

In [8]:
graph = {'a': {'b':14, 'c':9, 'd':7},
         'b': {'a':14, 'c':2, 'e':9},
         'c': {'a':9, 'b':2, 'd':10, 'f':11},
         'd': {'a':7, 'c':10, 'f':15},
         'e': {'b':9, 'f':6},
         'f': {'c':11, 'd':15, 'e':6}}

In [11]:
print('Use uniform edge distance:')
for vertex in graph.keys():
    dist = sg.dijkstra(graph, source=vertex, weight_dist_map=lambda x: 1)
    print(dist)
    
print('\nUse the original edge weight as edge distance:')
for vertex in graph.keys():
    dist = sg.dijkstra(graph, source=vertex)
    print(dist)
    
print('\nUse inverse of edge weight as edge distance:')
weight_dist_map = lambda x : 1. / x
for vertex in graph.keys():
    dist = sg.dijkstra(graph, source=vertex, weight_dist_map=weight_dist_map)
    print(dist)

Use uniform edge distance:
{'a': 0, 'b': 1, 'c': 1, 'd': 1, 'e': 2, 'f': 2}
{'b': 0, 'a': 1, 'c': 1, 'e': 1, 'd': 2, 'f': 2}
{'c': 0, 'a': 1, 'f': 1, 'b': 1, 'd': 1, 'e': 2}
{'d': 0, 'a': 1, 'f': 1, 'c': 1, 'b': 2, 'e': 2}
{'e': 0, 'b': 1, 'f': 1, 'a': 2, 'c': 2, 'd': 2}
{'f': 0, 'c': 1, 'd': 1, 'e': 1, 'b': 2, 'a': 2}

Use the original edge weight as edge distance:
{'a': 0, 'd': 7, 'c': 9, 'b': 11, 'f': 20, 'e': 20}
{'b': 0, 'c': 2, 'e': 9, 'a': 11, 'd': 12, 'f': 13}
{'c': 0, 'b': 2, 'a': 9, 'd': 10, 'e': 11, 'f': 11}
{'d': 0, 'a': 7, 'c': 10, 'b': 12, 'f': 15, 'e': 21}
{'e': 0, 'f': 6, 'b': 9, 'c': 11, 'a': 20, 'd': 21}
{'f': 0, 'e': 6, 'c': 11, 'b': 13, 'd': 15, 'a': 20}

Use inverse of edge weight as edge distance:
{'a': 0, 'b': 0.07142857142857142, 'c': 0.1111111111111111, 'd': 0.14285714285714285, 'e': 0.18253968253968253, 'f': 0.20202020202020202}
{'b': 0, 'a': 0.07142857142857142, 'e': 0.1111111111111111, 'c': 0.18253968253968253, 'd': 0.21428571428571427, 'f': 0.27344877344877

### Test 2: large random directed graph (takes a little bit long, so run only when necessary)

In [12]:
# num_vertices = 1000
# Erdos_Renyi_p = .1

# graph = random_directed_simple_graph(num_vertices, prob=Erdos_Renyi_p, random_weight=False)

# time0 = time.time()
# for vertex in graph.keys():
#     dist = dijkstra(graph, source=vertex)
#     # print(dist)
# time1 = time.time()

# print('Time elapsed: {:.3f} seconds'.format(time1 - time0))

## Load graph and calculate shortest path distance

In [13]:
import pickle

In [17]:
def load_graph_spd(filename, *, weight_dist_map=lambda x: x, log=True):
    
    """
    load graph and calculate shortest path distance
    """
    
    pickle_in = open(filename, 'rb')
    graph = pickle.load(pickle_in)
    
    if log:
        print('number of vertices = {}'.format(len(graph)))
        num_edges = 0
        for vertex, neighbors in graph.items():
            num_edges += len(neighbors)
        print('number of edges = {}'.format(num_edges))
    
    spd = {}
    time0 = time.time()
    for vertex in graph.keys():
        spd[vertex] = sg.dijkstra(graph, source=vertex, weight_dist_map=weight_dist_map)
    time1 = time.time()

    elapse = time1 - time0
    minutes = int(elapse / 60.)
    seconds = int(elapse - minutes * 60 + .5)
    
    if log:
        print('Time for calculating shortest path distances: {:d} minutes {:d} seconds'.format(minutes, seconds))
    
    return graph, spd

In [18]:
year = 1980
weight_dist_map = lambda x : 1. / x

### Undirected graph

In [19]:
filename = '../adjacency_list_{}_undirected'.format(year)
graph_und, spd_und = load_graph_spd(filename, weight_dist_map=weight_dist_map)

number of vertices = 1015
number of edges = 3416
Time for calculating shortest path distances: 0 minutes 21 seconds


### Directed graph

In [20]:
filename = '../adjacency_list_{}'.format(year)
graph_d, spd_d = load_graph_spd(filename, weight_dist_map=weight_dist_map)

number of vertices = 1015
number of edges = 2043
Time for calculating shortest path distances: 0 minutes 11 seconds


In [25]:
SCC = sg.stronglyConnectedComp(graph_d)
max_idx = 0
max_size = 0
for i, scc in enumerate(SCC):
    size = len(scc)
    if size > max_size:
        max_idx = i
        max_size = size
scc = SCC[max_idx]
print(len(scc))

355


In [27]:
subgraph = sg.induced_subgraph(graph_d, scc)

print('size of strongly connected component = {}'.format(len(scc)))
print('size of subgraph = {}'.format(len(subgraph)))

spd_sub = {}
time0 = time.time()
for vertex in subgraph.keys():
    spd_sub[vertex] = sg.dijkstra(subgraph, source=vertex, weight_dist_map=weight_dist_map)
time1 = time.time()

elapse = time1 - time0
minutes = int(elapse / 60.)
seconds = int(elapse - minutes * 60 + .5)

print('Time for calculating shortest path distances: {:d} minutes {:d} seconds'.format(minutes, seconds))

size of strongly connected component = 355
size of subgraph = 355
Time for calculating shortest path distances: 0 minutes 3 seconds


## Calculate curvature

In [27]:
# import sys
# sys.path.append('../Wasserstein')
# import Wasserstein_lp as wlp
import ot

In [32]:
def form_linprog_prob(graph, spd, head, tail, *, laziness=0, weight_prob_map=lambda x : x, inf_replace=1e7):
    
    """
    
    """
    
    nbh_head = set(graph[head].keys())
    nbh_tail = set(graph[tail].keys())
    nbh_joint = list(nbh_head.union(nbh_tail).union([head, tail]))
    nbh_joint_size = len(nbh_joint)
    idx_map = {nbh_joint[i]: i for i in range(nbh_joint_size)}

    # Random-walk distribution
    rwd_head = np.zeros(nbh_joint_size)
    rwd_tail = np.zeros(nbh_joint_size)

    rwd_head[idx_map[head]] = laziness
    rwd_tail[idx_map[tail]] = laziness

    for vertex, rwd in zip([head, tail], [rwd_head, rwd_tail]):
        total_weight = 0
        for neighbor in graph[vertex]:
            if neighbor != vertex:
                total_weight += weight_prob_map(graph[vertex][neighbor])
        for neighbor in graph[vertex]:
            if neighbor != vertex:
                rwd[idx_map[neighbor]] = (1. - laziness) * weight_prob_map(graph[vertex][neighbor]) / total_weight
        rwd /= sum(rwd)
        
    cost_matrix = np.zeros((nbh_joint_size, nbh_joint_size))
    
    for nb_source in nbh_joint:
        for nb_target in nbh_joint:
            if nb_target in spd[nb_source]:
                cost_matrix[idx_map[nb_source]][idx_map[nb_target]] = spd[nb_source][nb_target]
            else:
                cost_matrix[idx_map[nb_source]][idx_map[nb_target]] = inf_replace
    
    return rwd_head, rwd_tail, cost_matrix

In [36]:
def OR_curvature(graph, spd, *, laziness=0, log=True):
    edge_count = 0
    curvatures = {vertex: {} for vertex in graph.keys()}
    
    for head in graph:
        for tail in graph[head]:
            if head != tail:
                rwd_head, rwd_tail, cost_matrix = form_linprog_prob(graph, spd, head, tail, laziness=laziness)
                # if len(rwd_head) > 5:
                #     continue
                edge_count += 1
                gamma = ot.emd(rwd_head, rwd_tail, cost_matrix)#, reg=0.01)
                w_dist = np.dot(cost_matrix.flatten(), gamma.flatten())
                curvature = 1. - w_dist * graph[head][tail]
                curvatures[head][tail] = curvature
                
                if log:
                    print('number of edges calculated = {}'.format(edge_count))
                    print('\nedge: {} --> {}: size of joint neighborhood = {}'.format(head, tail, len(rwd_head)))
                    print('random walk at head =\n{}'.format(rwd_head))
                    print('random walk at tail =\n{}'.format(rwd_tail))
                    print('\ncost matrix = ')
                    for vec in cost_matrix:
                        for dpoint in vec:
                            print('{:.3f} '.format(dpoint), end='')
                        print()
                    print('\ngamma = ')
                    for vec in gamma:
                        for dpoint in vec:
                            print('{:.3f} '.format(dpoint), end='')
                        print()
                    print('W. distance = {}'.format(w_dist))
                    print('OR = {}'.format(curvature))
                    
    return curvatures

In [37]:
laziness = 0

In [38]:
curvatures_und = OR_curvature(graph_und, spd_und, laziness=laziness, log=False)

In [ ]:
curvature_d = OR_curvature(graph_d, spd_d, laziness=laziness, log=True)

number of edges calculated = 1

edge: 51201 --> 3859: size of joint neighborhood = 36
random walk at head =
[0.  0.  0.  0.2 0.  0.  0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0. ]
random walk at tail =
[0.05  0.025 0.025 0.    0.025 0.025 0.15  0.025 0.025 0.025 0.025 0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.    0.025 0.025 0.05  0.025
 0.025 0.025 0.025 0.025 0.025 0.025 0.025 0.    0.025 0.025 0.025 0.025]

cost matrix = 
0.000 2.000 2.000 1.000 2.000 2.000 0.500 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 1.500 2.000 2.000 1.000 2.000 1.500 1.500 2.000 2.000 2.000 2.000 2.000 2.000 2.000 2.000 1.000 2.000 2.000 2.000 2.000 
2.274 0.000 2.774 1.774 2.774 0.631 1.940 2.774 1.821 2.774 2.774 1.500 2.774 2.774 0.964 2.774 2.024 1.131 2.774 3.274 2.774 0.500 2.274 2.774 2.774 2.167 2.774 2.774 1.660 2.774 1.000 3.274 1.083 1.464 2.774 2.774 
4.333 3.683 0.000 3.833 4.833 2.933 4.000 4.833 3.881 

/software/python-3.6.1-el7-x86_64+intel-16.0/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/home/yhuang10/.local/lib/python3.6/site-packages/ot/lp/__init__.py:98: UserWarning: Problem infeasible. Check that a and b are in the simplex
  result_code_string = check_result(result_code)


2.000 2.000 2.000 1.500 1.581 2.000 2.000 1.083 2.000 2.000 0.000 2.000 2.000 1.917 2.000 2.000 1.490 2.000 1.200 2.000 2.593 2.000 1.500 1.000 1.157 1.343 1.843 1.500 0.917 1.847 1.139 
1.593 2.593 2.093 1.500 2.000 2.000 2.000 2.000 2.000 2.000 1.500 2.000 2.000 2.000 1.500 2.000 2.000 2.000 1.500 2.000 2.000 2.000 0.000 2.000 1.500 2.000 2.000 1.976 2.000 1.200 2.000 2.593 1.810 1.500 1.000 1.643 1.343 1.843 1.500 1.667 2.333 1.625 
2.950 3.950 3.300 2.681 3.633 3.633 3.633 3.633 3.633 3.181 2.300 3.633 3.633 3.633 3.133 3.248 3.633 3.633 2.633 3.633 3.633 3.300 3.633 0.000 3.200 3.633 3.633 3.157 3.633 2.833 3.633 3.950 3.450 3.133 2.633 2.824 2.700 3.200 3.133 2.200 3.514 2.710 
0.843 1.843 1.343 0.639 1.250 1.250 1.250 1.000 1.250 1.139 0.750 1.250 1.250 1.250 0.750 1.205 1.250 1.250 0.750 1.250 1.250 1.250 1.250 1.250 0.000 1.000 0.700 0.750 1.250 0.450 1.250 1.843 1.250 0.750 0.250 0.781 0.593 1.093 0.750 0.917 1.472 0.500 
3.543 4.543 4.043 3.143 3.950 3.950 3.950 3.700 3.950 

## Let us try on a small graph first 

In [39]:
graph = {'a': {'b':14, 'c':9, 'd':7},
         'b': {'a':14, 'c':2, 'e':9},
         'c': {'a':9, 'b':2, 'd':10, 'f':11},
         'd': {'a':7, 'c':10, 'f':15},
         'e': {'b':9, 'f':6},
         'f': {'c':11, 'd':15, 'e':6}}

spd = {}
weight_dist_map = lambda x : 1. / x
for vertex in graph.keys():
    dist = dijkstra(graph, source=vertex, weight_dist_map=weight_dist_map)
    spd[vertex] = dist
    print('{}'.format(vertex))
    for key, value in dist.items():
        print('\t{}: {:.3f}'.format(key, value))

a
	a: 0.000
	b: 0.071
	c: 0.111
	d: 0.143
	e: 0.183
	f: 0.202
b
	b: 0.000
	a: 0.071
	e: 0.111
	c: 0.183
	d: 0.214
	f: 0.273
c
	c: 0.000
	f: 0.091
	d: 0.100
	a: 0.111
	b: 0.183
	e: 0.258
d
	d: 0.000
	f: 0.067
	c: 0.100
	a: 0.143
	b: 0.214
	e: 0.233
e
	e: 0.000
	b: 0.111
	f: 0.167
	a: 0.183
	d: 0.233
	c: 0.258
f
	f: 0.000
	d: 0.067
	c: 0.091
	e: 0.167
	a: 0.202
	b: 0.273


In [49]:
laziness = .3

for head in graph:
    for tail in graph[head]:
        if head != tail:
            cost_matrix, rwd_head, rwd_tail = form_linprog_prob(graph, spd, head, tail, laziness)

            w_dist = wlp.Wasserstein_dist_lp(cost_matrix, rwd_head, rwd_tail).fun
            print('edge: {} -> {}\nsize of joint neighborhood = {}'.format(head, tail, len(rwd_head)))
            print('random walk at head =\n\t', end='')
            for dpoint in rwd_head:
                print('{:.3f} '.format(dpoint), end='')
            print('\nrandom walk at tail =\n\t', end='')
            for dpoint in rwd_tail:
                print('{:.3f} '.format(dpoint), end='')
            print('\ncost matrix = ')
            for vec in cost_matrix:
                print('\t', end='')
                for dpoint in vec:
                    print('{:.3f} '.format(dpoint), end='')
                print()
            print('W. distance = {:.3f}\n'.format(w_dist))

edge: a -> b
size of joint neighborhood = 5
random walk at head =
	0.000 0.210 0.163 0.300 0.327 
random walk at tail =
	0.252 0.056 0.000 0.392 0.300 
cost matrix = 
	0.000 0.258 0.233 0.183 0.111 
	0.258 0.000 0.100 0.111 0.183 
	0.233 0.100 0.000 0.143 0.214 
	0.183 0.111 0.143 0.000 0.071 
	0.111 0.183 0.214 0.071 0.000 
W. distance = 0.067

edge: a -> c
size of joint neighborhood = 5
random walk at head =
	0.210 0.163 0.300 0.000 0.327 
random walk at tail =
	0.300 0.219 0.197 0.241 0.044 
cost matrix = 
	0.000 0.100 0.111 0.091 0.183 
	0.100 0.000 0.143 0.067 0.214 
	0.111 0.143 0.000 0.202 0.071 
	0.091 0.067 0.202 0.000 0.273 
	0.183 0.214 0.071 0.273 0.000 
W. distance = 0.087

edge: a -> d
size of joint neighborhood = 5
random walk at head =
	0.210 0.163 0.300 0.000 0.327 
random walk at tail =
	0.219 0.300 0.153 0.328 0.000 
cost matrix = 
	0.000 0.100 0.111 0.091 0.183 
	0.100 0.000 0.143 0.067 0.214 
	0.111 0.143 0.000 0.202 0.071 
	0.091 0.067 0.202 0.000 0.273 
	0.183 0.